In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls = PyPDFLoader
    )

    document = loader.load()
    return document

In [3]:
import os

In [4]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\GEN_AI_BY_RP\\Medical-Chatbot-Using-Chainlit\\research'

In [10]:
%cd ..

c:\Users\Dell\OneDrive\Desktop\GEN_AI_BY_RP\Medical-Chatbot-Using-Chainlit


In [11]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\GEN_AI_BY_RP\\Medical-Chatbot-Using-Chainlit'

In [12]:
extracted_data = load_pdf_file(data="Data/")

In [13]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [14]:
text_chunks = text_split(extracted_data)


In [15]:
len(text_chunks)

8645

In [16]:
text_chunks

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637, 'page': 2, 'page_label': '3'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637, 'page': 3, 'page_label': '4'}, page_content='STAFF

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

In [18]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [19]:
embeddings = download_hugging_face_embeddings()

C:\Users\Dell\AppData\Local\Temp\ipykernel_26052\2604741814.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Dell\anaconda3\envs\MEDICAL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
query_result = embeddings.embed_query("Hello, world")


In [21]:
len(query_result)

384

In [22]:
query_result

[-0.056094296276569366,
 0.0355425700545311,
 0.004592856392264366,
 0.02386590465903282,
 -0.04944367706775665,
 -0.15514129400253296,
 0.06592562049627304,
 0.02249126322567463,
 -0.021727383136749268,
 0.014119219034910202,
 0.05505148693919182,
 0.0240557212382555,
 0.00501904496923089,
 -0.0064774444326758385,
 -0.034115586429834366,
 -0.05552099645137787,
 -0.006752697750926018,
 -0.023014001548290253,
 -0.17627857625484467,
 -0.02309209108352661,
 1.4231687600840814e-05,
 0.0793110728263855,
 -0.012627922929823399,
 0.03713003918528557,
 -0.09230004996061325,
 -0.02306787669658661,
 0.06069957837462425,
 0.05133027583360672,
 -0.02947750687599182,
 -0.03724546357989311,
 0.037288520485162735,
 0.05159961059689522,
 0.0963367372751236,
 -0.009374161250889301,
 -0.013310282491147518,
 0.0866255909204483,
 -0.08137478679418564,
 -0.06393181532621384,
 0.005632453132420778,
 0.0186680369079113,
 0.05008465424180031,
 -0.07173812389373779,
 -0.05522460490465164,
 -0.04673878103494644

In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GORQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [25]:
from pinecone import Pinecone, ServerlessSpec
import os

# Initialize Pinecone client
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")  # Ensure this is set in your environment
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define index name
index_name = "medicalbot2"

# Create the index
pc.create_index(
    name=index_name,
    dimension=384,  # Ensure this matches your embedding model's output dimension
    metric="cosine",  # Similarity metric
    spec=ServerlessSpec(  # Corrected class name
        cloud="aws",  # Cloud provider
        region="us-east-1"  # Region
    )
)


{
    "name": "medicalbot2",
    "metric": "cosine",
    "host": "medicalbot2-tuijg0f.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [26]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GORQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [27]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore(
    index_name = index_name,
    embedding = embeddings
)

In [28]:
vectorstore.add_documents(documents=text_chunks)

['6795ec33-c53d-4996-87e1-3d3178981e7e',
 '16934690-159b-4009-bee4-c2604c5ad725',
 '1f45a92d-888f-47ee-ad57-776d3a7c730d',
 '97292a8c-400d-4a2a-8be1-099123d127a8',
 'db9fb452-802d-4ac2-bc38-d7070cf42b5d',
 '2c866101-ba2f-4642-b4c4-1cfa74d729f8',
 '8b764e65-1eef-4fa3-901b-c71d08139fb9',
 '8aeffdfe-a293-4ebe-8158-ebc830c7d03c',
 '57c29555-4362-4eb4-954e-494ed42da419',
 'cb372bdc-6e30-42a0-ace3-7be7a4dd7f34',
 'c830dd92-2c64-46a6-9db8-922eabcd6459',
 'e9b648ad-bd0d-47e6-aae7-36a55584acbe',
 '7cfafc74-66f9-4255-afa4-fcd888b22b1f',
 'b5e34033-763d-42fe-a01f-e80a920c3465',
 '53a0d345-c463-49fa-b503-7e8874673137',
 'f5902743-d5f0-4e78-ae64-60faff1c7cda',
 '2aa5fb19-111b-4fec-920c-7756a7851605',
 'f4e33f63-b106-4ab1-b863-0ab6ff4c7799',
 '8d80d5d3-3365-4918-8d63-c95f1c8a48ba',
 '45b25b5b-dbc5-4514-97ed-dd9b94c9296e',
 '2ff3ee05-2fd3-4d37-8720-1c7f37bc50c8',
 '35810312-45c5-4c6b-a523-1cf27eb54130',
 '506dd240-c853-46a9-b2e7-3b86886d84ee',
 '7a7ad183-fb47-4b88-bacc-d84b916bb044',
 '3de9c261-b6dc-

In [29]:
retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs = {"k":3})

In [30]:
retrieved_docs = retriever.invoke("What is Acne?")

In [31]:
retrieved_docs

[Document(id='4a2e1722-b38b-4a7d-8dab-ba5c44382e4f', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 37.0, 'page_label': '38', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Acne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any\nage, it usually begins at puberty and worsens during ado-\nlescence. Nearly 85% of people develop acne at some time'),
 Document(id='c0d34c02-b1e0-4419-8ba0-8e73aef3e36b', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39'

In [32]:
!pip install langchain_groq

In [33]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0.4)

In [34]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [35]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

In [36]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [37]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})

In [38]:
response['answer']

'Acromegaly and gigantism are disorders caused by the abnormal release of a chemical from the pituitary gland in the brain, leading to increased growth in bone and soft tissue. Acromegaly occurs after bone growth stops, resulting in distinctive physical changes, while gigantism occurs before bone growth stops, causing unusual height. Both men and women can be affected, and diagnosis is often delayed due to the gradual onset of symptoms.'

In [39]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I don't have enough context to provide a complete answer, but I can tell you that "stats" is often short for "statistics," which is a branch of mathematics dealing with the collection, analysis, interpretation, and presentation of data.
